# EDA

In [1]:
import numpy as np
import pandas as pd

import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("mov_rec") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
rat_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/ratings.csv', header=True)
        

In [4]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [5]:
rat_rdd.tail(5)

[Row(userId=610, movieId=166534, rating=4.0, timestamp=1493848402),
 Row(userId=610, movieId=168248, rating=5.0, timestamp=1493850091),
 Row(userId=610, movieId=168250, rating=5.0, timestamp=1494273047),
 Row(userId=610, movieId=168252, rating=5.0, timestamp=1493846352),
 Row(userId=610, movieId=170875, rating=3.0, timestamp=1493846415)]

In [6]:
rat_rdd.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
mov_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/movies.csv', header=True)
        

In [8]:
# tags_rdd = spark.read.format('com.databricks.spark.csv').\
#                              options(header='true',\
#                              inferschema='true').\
#                              load('data/ml-latest-small/tags.csv', header=True)
        

In [9]:
mov_rdd.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [10]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [11]:
mov_rdd.createOrReplaceTempView("mov_rdd")
rat_rdd.createOrReplaceTempView("rat_rdd")

In [12]:
# tags_rdd.show(5)

Leave out timestamp...

In [13]:
mov_surprise = spark.sql(
'''
SELECT ratings.userId, movies.title, ratings.rating 
  FROM mov_rdd AS movies
  JOIN rat_rdd AS ratings
    ON movies.movieId = ratings.movieId
'''
)

In [14]:
mov_surprise.createOrReplaceTempView("mov_surprise")

In [15]:
mov_surprise.show(5)

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|     1|    Toy Story (1995)|   4.0|
|     1|Grumpier Old Men ...|   4.0|
|     1|         Heat (1995)|   4.0|
|     1|Seven (a.k.a. Se7...|   5.0|
|     1|Usual Suspects, T...|   5.0|
+------+--------------------+------+
only showing top 5 rows



# Similarity Based Recommender using Surprise

Begin by reimplementing the similarity based recommender.

In [16]:
# # install surprise if not already available
# pip install surprise

In [17]:
from surprise import KNNBasic, KNNWithMeans, SVD

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate

# mov_surprise doesn't contain any zero values
mov_surprise = mov_surprise[mov_surprise['rating']!=0]

# ratings in dataset are from 0.5 to 5
reader = Reader(rating_scale=(0.5, 5))

In [18]:
mov_surprise.head(5)

[Row(userId=1, title='Toy Story (1995)', rating=4.0),
 Row(userId=1, title='Grumpier Old Men (1995)', rating=4.0),
 Row(userId=1, title='Heat (1995)', rating=4.0),
 Row(userId=1, title='Seven (a.k.a. Se7en) (1995)', rating=5.0),
 Row(userId=1, title='Usual Suspects, The (1995)', rating=5.0)]

In [19]:
# convert to pandas DataFrame
# warning, if file is too large, this will likely crash...
mov_pDF = mov_surprise.toPandas()
print(mov_pDF.head(5))

surprise_movie_ratings = Dataset.load_from_df(mov_pDF, reader)

   userId                        title  rating
0       1             Toy Story (1995)     4.0
1       1      Grumpier Old Men (1995)     4.0
2       1                  Heat (1995)     4.0
3       1  Seven (a.k.a. Se7en) (1995)     5.0
4       1   Usual Suspects, The (1995)     5.0


In [20]:
print(mov_pDF.tail(5))

        userId                           title  rating
100831     610                    Split (2017)     4.0
100832     610   John Wick: Chapter Two (2017)     5.0
100833     610                  Get Out (2017)     5.0
100834     610                    Logan (2017)     5.0
100835     610  The Fate of the Furious (2017)     3.0


In [21]:
# treat data as training set
training_movie_data = surprise_movie_ratings.build_full_trainset()

# anything not a user-item rating pair as test
# no data for this, so we can't do an eval on out of sample accuracy
testing_movie_data = training_movie_data.build_anti_testset()

# create instance of recommender algorithm object
# algo = KNNBasic()
algo = SVD()

# fit it on the training data
algo.fit(training_movie_data)

# predict on the test data
predictions = algo.test(testing_movie_data)

In [21]:
# print the first 5 predictions
predictions[:5]

[Prediction(uid=1, iid='Shawshank Redemption, The (1994)', r_ui=3.501556983616962, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid='Good Will Hunting (1997)', r_ui=3.501556983616962, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid='Kill Bill: Vol. 1 (2003)', r_ui=3.501556983616962, est=4.593419317376364, details={'was_impossible': False}),
 Prediction(uid=1, iid='Collateral (2004)', r_ui=3.501556983616962, est=4.621110157379944, details={'was_impossible': False}),
 Prediction(uid=1, iid='Talladega Nights: The Ballad of Ricky Bobby (2006)', r_ui=3.501556983616962, est=4.103684136782077, details={'was_impossible': False})]

In [22]:
# print last 5 predictions
predictions[-5:]

[Prediction(uid=610, iid='United States of Leland, The (2003)', r_ui=3.501556983616962, est=3.4483464512846345, details={'was_impossible': False}),
 Prediction(uid=610, iid='A Home at the End of the World (2004)', r_ui=3.501556983616962, est=3.9013496548364768, details={'was_impossible': False}),
 Prediction(uid=610, iid='League of Ordinary Gentlemen, A (2004)', r_ui=3.501556983616962, est=3.737684618942152, details={'was_impossible': False}),
 Prediction(uid=610, iid='I Think I Love My Wife (2007)', r_ui=3.501556983616962, est=3.372100516495243, details={'was_impossible': False}),
 Prediction(uid=610, iid='Man of the Year (1995)', r_ui=3.501556983616962, est=3.4610365171213604, details={'was_impossible': False})]

## Cross Validate: SVD

In [23]:
# standard format for cross validation
# algorithm used, ratings dataset, metrics, number of cross validations, verbose controls how much output
cross_validate(SVD(n_factors=5, reg_all=0.005), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8745  0.8697  0.8599  0.8681  0.8798  0.8704  0.0067  
MAE (testset)     0.6711  0.6690  0.6620  0.6652  0.6770  0.6689  0.0051  
Fit time          2.21    2.22    2.30    2.33    2.50    2.31    0.11    
Test time         0.24    0.23    0.23    0.24    0.22    0.23    0.01    


{'test_rmse': array([0.87453864, 0.86972837, 0.85991258, 0.86805887, 0.87978943]),
 'test_mae': array([0.67114805, 0.66901224, 0.66200095, 0.66515867, 0.67698563]),
 'fit_time': (2.2120163440704346,
  2.219325304031372,
  2.3049275875091553,
  2.330012559890747,
  2.503704071044922),
 'test_time': (0.24231410026550293,
  0.22815513610839844,
  0.2279045581817627,
  0.23625707626342773,
  0.22142314910888672)}

## Cross Validtate: KNNBasic

In [24]:
cross_validate(KNNBasic(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9661  0.9666  0.9558  0.9499  0.9532  0.9583  0.0068  
MAE (testset)     0.7342  0.7363  0.7284  0.7268  0.7271  0.7306  0.0039  
Fit time          0.13    0.14    0.15    0.15    0.15    0.14    0.01    
Test time         1.59    1.59    1.53    1.63    1.53    1.58    0.04    


{'test_rmse': array([0.9660555 , 0.96657213, 0.95578539, 0.94993649, 0.95315412]),
 'test_mae': array([0.73420952, 0.73634307, 0.72838913, 0.72682531, 0.72706225]),
 'fit_time': (0.13243579864501953,
  0.14374589920043945,
  0.14587783813476562,
  0.1508474349975586,
  0.14685869216918945),
 'test_time': (1.5865395069122314,
  1.5939366817474365,
  1.5339577198028564,
  1.6309318542480469,
  1.5313568115234375)}

## Cross Validate: KNNWithMeans

In [25]:
cross_validate(KNNWithMeans(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9343  0.9214  0.9241  0.9368  0.9298  0.9293  0.0058  
MAE (testset)     0.7181  0.7111  0.7113  0.7175  0.7173  0.7151  0.0032  
Fit time          0.16    0.18    0.19    0.17    0.18    0.18    0.01    
Test time         1.61    1.57    1.55    1.67    1.71    1.62    0.06    


{'test_rmse': array([0.93428011, 0.92143209, 0.92413232, 0.93676997, 0.92977712]),
 'test_mae': array([0.71805746, 0.71108627, 0.71133904, 0.71752862, 0.71730823]),
 'fit_time': (0.16377639770507812,
  0.18019533157348633,
  0.18958735466003418,
  0.17432522773742676,
  0.18480873107910156),
 'test_time': (1.6059725284576416,
  1.5661771297454834,
  1.5508019924163818,
  1.670332908630371,
  1.7058041095733643)}

# Comparing to Baseline Model

In [26]:
from src/baselines import 


mov_pDF.head()

SyntaxError: invalid syntax (<ipython-input-26-e4aed93defeb>, line 1)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8721  0.8689  0.8769  0.8609  0.8676  0.8693  0.0053  
MAE (testset)     0.6697  0.6674  0.6752  0.6618  0.6667  0.6682  0.0044  
Fit time          3.00    2.77    2.96    2.80    2.67    2.84    0.12    
Test time         0.35    0.36    0.31    0.30    0.26    0.32    0.04    
{'test_rmse': array([0.87209268, 0.86891766, 0.87692285, 0.86086742, 0.86756916]),
 'test_mae': array([0.66973578, 0.66740167, 0.67518763, 0.66177259, 0.66667806]),
 'fit_time': (3.004497766494751,
  2.7695112228393555,
  2.956921100616455,
  2.801307201385498,
  2.6742496490478516),
 'test_time': (0.34892988204956055,
  0.36215710639953613,
  0.3130302429199219,
  0.3012969493865967,
  0.26192665100097656)}